## Import danych

#### Użyte Biblioteki i ich wersje:
- python: 3.12.7
- pandas: 2.2.3
- spaCy: 3.8.5
- nltk: 3.9.1
- emoji: 2.14.1
- model pl_core_news_lg : 3.8.0

In [1]:
#import potrzebnych bibliotek

import pandas as pd
import os

In [2]:
folder_path = r"../data/data_14.04-05.05"


#zaciągnięcie wszystkich plikow z tweetami do ramki danych
all_dfs = []

for file in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    df["source_file"] = file  #dodanie kolumny z nazwą pliku
    all_dfs.append(df)



combined_df = pd.concat(all_dfs)
combined_df.sample(5)


,id,url,content,author,created_at,source_file
20,1916271784170553480,https://x.com/AwZbaw/status/1916271784170553480,Głosuj na Rafała #NormalnaPolska z #Trzaskowsk...,AwZbaw,Sat Apr 26 23:22:50 +0000 2025,tweets__Trzaskowski2025_20250505_121152.csv
761,1916911763372527664,https://x.com/mdw_warszawa_/status/19169117633...,Kto zrobił to lepiej? #debata #Mentzen2025 #Me...,mdw_warszawa_,Mon Apr 28 17:45:53 +0000 2025,tweets__Mentzen2025_20250505_111555.csv
24,1919102074949632065,https://x.com/KSzymanskiKonf/status/1919102074...,Zachęcam do obejrzenia mojego dzisiejszego prz...,KSzymanskiKonf,Sun May 04 18:49:24 +0000 2025,tweets__Mentzen2025_20250505_111555.csv
719,1919077947442418056,https://x.com/bebelilllly/status/1919077947442...,Bo Kobiety są siła i trzaskają.Nie dla przedmi...,bebelilllly,Sun May 04 17:13:32 +0000 2025,tweets__Trzaskowski2025_20250505_122839.csv
851,1916772939405344805,https://x.com/Mateusz_PLN/status/1916772939405...,"@t_stanislawski Tylko, że to PO i PiS nakręcaj...",Mateusz_PLN,Mon Apr 28 08:34:15 +0000 2025,tweets__Mentzen2025_20250505_111555.csv


In [3]:
#ilość tweetow przed oczyszczeniem

print(combined_df['id'].nunique()) #ilość unikalnych tweetów
print(combined_df['id'].count())   #ilość wszystkich tweetów

7552
7719


In [4]:
#wstepna transformacja ramki danych
combined_df.rename(columns={'content': 'tweet'}, inplace=True)

def trim_filename(filename):
    return '_'.join(filename.split('_')[:3])

combined_df['hasztag/canidate'] = combined_df['source_file'].apply(trim_filename)

#usunięcie duplikatow tweetow
combined_df = combined_df.drop_duplicates(subset='id', keep='first')

In [5]:
# ilość unikalnych tweetow po usunięciu duplikatów

print(combined_df['id'].nunique())
print(combined_df['id'].count())

7552
7552


## Wyczyszczenie danych

In [6]:
#zaimportowanie potrzebnych bibliotek

import re
import spacy
import nltk
import emoji

In [7]:
#funkcja, oczyszczająca tweety

def clean_tweets(tweet):
    if not isinstance(tweet, str):
        return ""

    #usuniecie adresow e-mail
    tweet = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', ' ', tweet)

    #usuniecie url
    tweet = re.sub(r"http\S+|www\.\S+", ' ', tweet)

    #usuniecie RT
    tweet = re.sub(r'\bRT @\w+', ' ' , tweet)

    #usuniecie hasztagow
    tweet = re.sub(r'#', ' ', tweet)

    #usuniecie user tagów
    tweet = re.sub(r'@\w+', ' ', tweet)

    #usuniecie html tagów
    tweet = re.sub(r'<.*?>', ' ', tweet)

    #usuniecie powtorzonych zankow
    tweet = re.sub(r'(.)\1+', r'\1', tweet)

    #zmiana na male litery
    tweet = tweet.lower()

    #usuniecie emotek
    tweet = emoji.replace_emoji(tweet, " ")

    return tweet

combined_df['tweet_clean'] = combined_df['tweet'].apply(clean_tweets)

#usuniecie wierszy, gdzie 'tweet_clean' jest puste lub jest tylko spacją
combined_df=combined_df[combined_df['tweet_clean'].str.strip() != '']

In [8]:
#ilosc tweetow po usunieciu pustych wierszy
print(combined_df['id'].count())
combined_df.sample(10)

7440


,id,url,tweet,author,created_at,source_file,hasztag/canidate,tweet_clean
162,1911911312612794874,https://x.com/batkmano/status/1911911312612794874,"Może i śmieszne, ale bardzo celne stwierdzenie...",batkmano,Mon Apr 14 22:35:53 +0000 2025,tweets__DebataPrezydencka_20250505_122438.csv,tweets__DebataPrezydencka,"może i śmieszne, ale bardzo celne stwierdzenie..."
211,1918609686514844071,https://x.com/pumpmaster2021/status/1918609686...,@NawrockiKn Biegniesz w czapce #Mentzen2025 ??,pumpmaster2021,Sat May 03 10:12:49 +0000 2025,tweets__Mentzen2025_20250505_111555.csv,tweets__Mentzen2025,biegniesz w czapce mentzen2025 ?
245,1916209982615523572,https://x.com/PiotrKrzyszto10/status/191620998...,Wybieram Was #Trzaskowski2025 https://t.co/6Pv...,PiotrKrzyszto10,Sat Apr 26 19:17:15 +0000 2025,tweets__Trzaskowski2025_20250505_121152.csv,tweets__Trzaskowski2025,wybieram was trzaskowski2025
2,1918969515330662728,https://x.com/AktualniePL/status/1918969515330...,"@AktualniePL 🗳️ Debata prezydencka ""Bunt Młody...",AktualniePL,Sun May 04 10:02:39 +0000 2025,tweets__DebataPrezydencka_20250505_114856.csv,tweets__DebataPrezydencka,"debata prezydencka ""bunt młodych"" już dziś ..."
25,1912571963173724558,https://x.com/PiotrSiwek13/status/191257196317...,#wybory #wybory2025 #Trzaskowski2025 #Trzaskam...,PiotrSiwek13,Wed Apr 16 18:21:04 +0000 2025,tweets__DebataPrezydencka_20250505_122438.csv,tweets__DebataPrezydencka,wybory wybory2025 trzaskowski2025 trzaskamypi...
9,1919280101553840486,https://x.com/WaldMar72/status/191928010155384...,@SlawomirMentzen Na ostatnim zebraniu Spółdzie...,WaldMar72,Mon May 05 06:36:49 +0000 2025,tweets__Mentzen_replies_1919057413300179189_20...,tweets__Mentzen,na ostatnim zebraniu spółdzieli mieszkaniowej...
152,1916918496623026394,https://x.com/Zamoyski1/status/191691849662302...,#debata Głębokie prace polowe @NawrockiKn #Wyb...,Zamoyski1,Mon Apr 28 18:12:38 +0000 2025,tweets__WyboryPrezydenckie_20250505_115906.csv,tweets__WyboryPrezydenckie,debata głębokie prace polowe wybory2025 wybor...
161,1912802751467135353,https://x.com/wiosnakobiet/status/191280275146...,Czy wreszcie doczekamy się ukarania Brauna za ...,wiosnakobiet,Thu Apr 17 09:38:08 +0000 2025,tweets__Biejat2025_20250505_122318.csv,tweets__Biejat2025,czy wreszcie doczekamy się ukarania brauna za ...
34,1915331432014135425,https://x.com/Micha81842136/status/19153314320...,"Potrzebuje pomocy w pracy, nie jest to sondaż,...",Micha81842136,Thu Apr 24 09:06:13 +0000 2025,tweets__zandberg2025_20250505_120449.csv,tweets__zandberg2025,"potrzebuje pomocy w pracy, nie jest to sondaż,..."
161,1916811863741866383,https://x.com/GabaLenartowicz/status/191681186...,📊 Najnowszy sondaż IBRiS dla Polskiego Radia 2...,GabaLenartowicz,Mon Apr 28 11:08:55 +0000 2025,tweets__WyboryPrezydenckie_20250505_115906.csv,tweets__WyboryPrezydenckie,najnowszy sondaż ibris dla polskiego radia 2...


## Usuniecie Stop Words

In [10]:
#zaimportowanie modelu do przetwarzania danych w języku polskim
import pl_core_news_lg
nlp = pl_core_news_lg.load()

stop_words = nlp.Defaults.stop_words

combined_df['tweet_no_stop'] = combined_df['tweet_clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
combined_df

,id,url,tweet,author,created_at,source_file,hasztag/canidate,tweet_clean,tweet_no_stop
0,1919133406559862822,https://x.com/MikulskiMa43709/status/191913340...,@ZandbergRAZEM chlopie nie dosc ze skopiowales...,MikulskiMa43709,Sun May 04 20:53:54 +0000 2025,tweetes__Zandberg_replies_1918755129492115523_...,tweetes__Zandberg,chlopie nie dosc ze skopiowales od mentzena u...,chlopie skopiowales mentzena ulotki same sztan...
1,1919119684839882797,https://x.com/AKaraskiewicz/status/19191196848...,@ZandbergRAZEM Skąd pieniądze komunizujący soc...,AKaraskiewicz,Sun May 04 19:59:23 +0000 2025,tweetes__Zandberg_replies_1918755129492115523_...,tweetes__Zandberg,skąd pieniądze komunizujący socjalisto? z czt...,pieniądze komunizujący socjalisto? czterodniow...
2,1919111134520442899,https://x.com/Krystianro/status/19191111345204...,@NGoldschneider @Adam32330707 @ZandbergRAZEM N...,Krystianro,Sun May 04 19:25:24 +0000 2025,tweetes__Zandberg_replies_1918755129492115523_...,tweetes__Zandberg,nie wiedziałem tego.,wiedziałem tego.
3,1919102567839125857,https://x.com/pawe_semmler/status/191910256783...,@ZandbergRAZEM Jesteś żałosny..🤦,pawe_semmler,Sun May 04 18:51:22 +0000 2025,tweetes__Zandberg_replies_1918755129492115523_...,tweetes__Zandberg,jesteś żałosny.,jesteś żałosny.
4,1919071945691103433,https://x.com/Dorota_2017/status/1919071945691...,"@ZandbergRAZEM Nieśmiało przypomnę panu, że po...",Dorota_2017,Sun May 04 16:49:41 +0000 2025,tweetes__Zandberg_replies_1918755129492115523_...,tweetes__Zandberg,"nieśmiało przypomnę panu, że poparcie pana i ...","nieśmiało przypomnę panu, poparcie parti razem..."
...,...,...,...,...,...,...,...,...,...
11,1918614180887728637,https://x.com/Anyone1617/status/19186141808877...,@ZandbergRAZEM A Manhattan?,Anyone1617,Sat May 03 10:30:41 +0000 2025,tweets__Zandberg_replies_1918389965593166053_2...,tweets__Zandberg,a manhatan?,manhatan?
12,1918612645936030069,https://x.com/Izabela12671885/status/191861264...,@ZandbergRAZEM Gdybyś jakimś cudem.. dostał si...,Izabela12671885,Sat May 03 10:24:35 +0000 2025,tweets__Zandberg_replies_1918389965593166053_2...,tweets__Zandberg,gdybyś jakimś cudem. dostał się do drugiej tu...,gdybyś jakimś cudem. dostał drugiej tury wybor...
13,1918589284098945258,https://x.com/Piotr_ibin_Alab/status/191858928...,"@ZandbergRAZEM Adrian, ile za tę niepotrzebna ...",Piotr_ibin_Alab,Sat May 03 08:51:45 +0000 2025,tweets__Zandberg_replies_1918389965593166053_2...,tweets__Zandberg,"adrian, ile za tę niepotrzebna kampanię można...","adrian, niepotrzebna kampanię sfinansować mies..."
14,1918583298600669283,https://x.com/grandesie/status/191858329860066...,@ZandbergRAZEM Panie Adrianie dlaczego spotkan...,grandesie,Sat May 03 08:27:58 +0000 2025,tweets__Zandberg_replies_1918389965593166053_2...,tweets__Zandberg,panie adrianie dlaczego spotkanie w wolsztyni...,panie adrianie spotkanie wolsztynie 9:15? zain...


## Lematyzacja

In [11]:
#lemmatyzacja

def lemmatize(tweet):
    tweet = nlp(tweet)
    tokens = []
    for token in tweet:
        tokens.append(token.lemma_)

    tweet = ' '.join(tokens)

    #usuniecie nadmiernych spacji
    pattern = r'\s+([,.!?;:])'
    tweet = re.sub(pattern, r'\1', tweet)
    tweet = re.sub(r'\s+', ' ', tweet)
    tweet = tweet.strip()
    return tweet



combined_df['tweet_base_no_stop'] = combined_df['tweet_no_stop'].apply(lemmatize)
combined_df

,id,url,tweet,author,created_at,source_file,hasztag/canidate,tweet_clean,tweet_no_stop,tweet_base_no_stop
0,1919133406559862822,https://x.com/MikulskiMa43709/status/191913340...,@ZandbergRAZEM chlopie nie dosc ze skopiowales...,MikulskiMa43709,Sun May 04 20:53:54 +0000 2025,tweetes__Zandberg_replies_1918755129492115523_...,tweetes__Zandberg,chlopie nie dosc ze skopiowales od mentzena u...,chlopie skopiowales mentzena ulotki same sztan...,chlop skopiowales mentzena ulotka sam sztandar...
1,1919119684839882797,https://x.com/AKaraskiewicz/status/19191196848...,@ZandbergRAZEM Skąd pieniądze komunizujący soc...,AKaraskiewicz,Sun May 04 19:59:23 +0000 2025,tweetes__Zandberg_replies_1918755129492115523_...,tweetes__Zandberg,skąd pieniądze komunizujący socjalisto? z czt...,pieniądze komunizujący socjalisto? czterodniow...,pieniądz komunizujący socjalisto? czterodniowy...
2,1919111134520442899,https://x.com/Krystianro/status/19191111345204...,@NGoldschneider @Adam32330707 @ZandbergRAZEM N...,Krystianro,Sun May 04 19:25:24 +0000 2025,tweetes__Zandberg_replies_1918755129492115523_...,tweetes__Zandberg,nie wiedziałem tego.,wiedziałem tego.,wiedzieć być to.
3,1919102567839125857,https://x.com/pawe_semmler/status/191910256783...,@ZandbergRAZEM Jesteś żałosny..🤦,pawe_semmler,Sun May 04 18:51:22 +0000 2025,tweetes__Zandberg_replies_1918755129492115523_...,tweetes__Zandberg,jesteś żałosny.,jesteś żałosny.,być żałosny.
4,1919071945691103433,https://x.com/Dorota_2017/status/1919071945691...,"@ZandbergRAZEM Nieśmiało przypomnę panu, że po...",Dorota_2017,Sun May 04 16:49:41 +0000 2025,tweetes__Zandberg_replies_1918755129492115523_...,tweetes__Zandberg,"nieśmiało przypomnę panu, że poparcie pana i ...","nieśmiało przypomnę panu, poparcie parti razem...","nieśmiało przypomnieć pan, poparcie partia raz..."
...,...,...,...,...,...,...,...,...,...,...
11,1918614180887728637,https://x.com/Anyone1617/status/19186141808877...,@ZandbergRAZEM A Manhattan?,Anyone1617,Sat May 03 10:30:41 +0000 2025,tweets__Zandberg_replies_1918389965593166053_2...,tweets__Zandberg,a manhatan?,manhatan?,Manhatan?
12,1918612645936030069,https://x.com/Izabela12671885/status/191861264...,@ZandbergRAZEM Gdybyś jakimś cudem.. dostał si...,Izabela12671885,Sat May 03 10:24:35 +0000 2025,tweets__Zandberg_replies_1918389965593166053_2...,tweets__Zandberg,gdybyś jakimś cudem. dostał się do drugiej tu...,gdybyś jakimś cudem. dostał drugiej tury wybor...,gdyby być jakiś cud. dostać drugi tura wyborów...
13,1918589284098945258,https://x.com/Piotr_ibin_Alab/status/191858928...,"@ZandbergRAZEM Adrian, ile za tę niepotrzebna ...",Piotr_ibin_Alab,Sat May 03 08:51:45 +0000 2025,tweets__Zandberg_replies_1918389965593166053_2...,tweets__Zandberg,"adrian, ile za tę niepotrzebna kampanię można...","adrian, niepotrzebna kampanię sfinansować mies...","adrian, niepotrzebny kampania sfinansować mies..."
14,1918583298600669283,https://x.com/grandesie/status/191858329860066...,@ZandbergRAZEM Panie Adrianie dlaczego spotkan...,grandesie,Sat May 03 08:27:58 +0000 2025,tweets__Zandberg_replies_1918389965593166053_2...,tweets__Zandberg,panie adrianie dlaczego spotkanie w wolsztyni...,panie adrianie spotkanie wolsztynie 9:15? zain...,pani adrianąć spotkanie wolsztynie 9:15? zaint...


In [24]:
#wyeksportowanie danych do CSV

def export_to_csv(df, filename, path):
    full_path = f"{path}/{filename}"
    df.to_csv(full_path, index=False)

export_to_csv(df = combined_df, filename= 'clean_data 14.04-05.05.csv',  path="../data")